# Wines Points prediction 

Submission Date : 3.6.2023
Task: Predict the wine score given the inputs
Instructions:
 * Use logistic regression as benchmark model
 * Use sklearn pipeliens + cv + grid search with sklearn models (e.g. KNNs, RandomForest, etc.)
 * Compare all models on proper metric (your choice)
 #sklearn machine learning

For DNN course project:
* Use sklearn pipeliens with tensorflow models (w/wo embeddings, LSTMs, RNNs, Transformers etc.)
* Compare all models on proper metric (your choice)

In [15]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Here we will try to predict the points a wine will get based on known characteristics (i.e. features, in the ML terminology). The mine point in this stage is to establish a simple, ideally super cost effective, basline.
In the real world there is a tradeoff between complexity and perforamnce, and the DS job, among others, is to present a tradeoff tables of what performance is achivalbel at what complexity level. 

to which models with increased complexity and resource demands will be compared. Complexity should then be translated into cost. For example:
 * Compute cost 
 * Maintenance cost
 * Serving costs (i.e. is new platform needed?) 
 

## Loading the data

In [16]:
import pandas as pd
import cufflinks as cf; cf.go_offline()
from sklearn.model_selection import train_test_split

In [17]:
import os
import numpy as np
import seaborn as sns
from sklearn import metrics
import plotly.express as px
import matplotlib.pyplot as plt
import sys; sys.path.append('../')
from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.linear_model import LogisticRegression ,LinearRegression
from sklearn.neighbors import KNeighborsClassifier , KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler , OneHotEncoder , LabelEncoder , StandardScaler
%matplotlib inline

In [18]:
wine_reviews = pd.read_csv("C:/Users/USER\Python_ML_2023/data/winemag-data-130k-v2.csv")
wine_reviews.shape

(129971, 14)

In [19]:
wine_reviews.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [20]:
wine_reviews.nunique()

Unnamed: 0               129971
country                      43
description              119955
designation               37979
points                       21
price                       390
province                    425
region_1                   1229
region_2                     17
taster_name                  19
taster_twitter_handle        15
title                    118840
variety                     707
winery                    16757
dtype: int64

In [21]:
wine_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


In [22]:
#1 Data Preparation:
wine_reviews.drop(['taster_name','taster_twitter_handle','designation','description','region_2','region_1'],axis=1, inplace=True)
wine_reviews.drop(columns=wine_reviews.columns[0], axis=1,  inplace=True)
wine_reviews.head(5)

,country,points,price,province,title,variety,winery
0,Italy,87,NaN,Sicily & Sardinia,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,87,15.0,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,87,14.0,Oregon,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,87,13.0,Michigan,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,87,65.0,Oregon,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [23]:
print(wine_reviews.shape)
wine_reviews.isnull().sum()

(129971, 7)


country       63
points         0
price       8996
province      63
title          0
variety        1
winery         0
dtype: int64

In [24]:
#dropping catagories with small counts
# province drop
counts = wine_reviews['province'].value_counts()
filtered = counts[counts < 10].index
wine_reviews = wine_reviews[~wine_reviews['province'].isin(filtered)]

# country drop
counts = wine_reviews['country'].value_counts()
filtered = counts[counts < 10].index
wine_reviews = wine_reviews[~wine_reviews['country'].isin(filtered)]

# variety drop
counts = wine_reviews['variety'].value_counts()
filtered = counts[counts < 10].index
wine_reviews = wine_reviews[~wine_reviews['variety'].isin(filtered)]

# winery drop
counts = wine_reviews['winery'].value_counts()
filtered = counts[counts < 10].index
wine_reviews = wine_reviews[~wine_reviews['winery'].isin(filtered)]

In [25]:
avg_points_by_country = wine_reviews.groupby(['country','province']).points.mean()
avg_points_by_country.head(10)

country    province         
Argentina  Mendoza Province     87.142668
           Other                86.059375
Australia  Australia Other      85.742268
           New South Wales      88.500000
           South Australia      89.580132
           Tasmania             91.500000
           Victoria             88.915493
           Western Australia    89.463687
Austria    Austria              87.900000
           Burgenland           89.557730
Name: points, dtype: float64

In [26]:
avg_points_by_price = wine_reviews.groupby('price').points.mean()
avg_points_by_price.tail(10)

price
900.0      94.0
932.0      97.0
973.0      95.0
980.0      94.0
1000.0     95.5
1100.0     95.5
1200.0     96.0
1300.0     96.0
1500.0    100.0
1900.0     98.0
Name: points, dtype: float64

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

# Drop any rows with missing values in the 'title' and 'points' columns
data = wine_reviews.dropna(subset=['title', 'points'])

# Select the 'title' and 'points' columns for correlation analysis
title = data['title']
points = data['points']

# Create an instance of CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the 'title' column using CountVectorizer
encoded_data = vectorizer.fit_transform(title)

# Convert the sparse matrix to a DataFrame
encoded_df = pd.DataFrame.sparse.from_spmatrix(encoded_data, columns=vectorizer.get_feature_names_out())

# Concatenate the DataFrame with the 'points' column
data_encoded = pd.concat([encoded_df, points], axis=1)

# Calculate the correlation between the encoded features and 'points'
correlation = data_encoded.corr()['points'].drop('points')

# Print the correlation values
print(correlation)
if all(correlation > -0.2) or all(correlation < 0.2):
    print("Uncorrelated")
else:
    print("Correlated")

MemoryError: Unable to allocate 15.8 GiB for an array with shape (18156, 116724) and data type float64

In [12]:
#splitting the Dataand Feature Selection:
X = wine_reviews.drop('points', axis=1)
y = wine_reviews['points']
print(y.min())
print(y.max())

80
100


In [13]:
X_train , X_test , y_train , y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Model Training: 
def metrics_check(y_test,y_pred):
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    print('R-squared:', metrics.r2_score(y_test, y_pred))

In [ ]:
#Model Evaluation: logistic regression model. 

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)

metrics_check(y_test,y_pred)